In [1]:
from make_dataset import MultiMatchSoccerDataset

In [2]:
dataset = MultiMatchSoccerDataset(data_root="match_data")

Loading Matches: 100%|██████████| 6/6 [00:41<00:00,  6.91s/it]


In [3]:
sample = dataset[156]
sample["graph"]

HeteroData(
  Node={ x=[2875, 10] },
  (Node, attk_and_attk, Node)={
    edge_index=[2, 7782],
    edge_attr=[7782, 1],
  },
  (Node, attk_and_def, Node)={
    edge_index=[2, 13848],
    edge_attr=[13848, 1],
  },
  (Node, def_and_def, Node)={
    edge_index=[2, 3682],
    edge_attr=[3682, 1],
  },
  (Node, attk_and_ball, Node)={
    edge_index=[2, 1362],
    edge_attr=[1362, 1],
  },
  (Node, def_and_ball, Node)={
    edge_index=[2, 1252],
    edge_attr=[1252, 1],
  },
  (Node, temporal, Node)={
    edge_index=[2, 2852],
    edge_attr=[2852, 1],
  }
)

In [4]:
import torch
graph = sample["graph"]

In [5]:
x = graph['Node'].x
print(x.min(), x.max())
for rel in graph.edge_index_dict:
    w = graph.edge_attr_dict[rel].view(-1)
    print(rel, w.min(), w.max())

tensor(-1.) tensor(22.)
('Node', 'attk_and_attk', 'Node') tensor(0.1000) tensor(2.0000)
('Node', 'attk_and_def', 'Node') tensor(1.0000e-06) tensor(4.0366)
('Node', 'def_and_def', 'Node') tensor(0.1001) tensor(1.0000)
('Node', 'attk_and_ball', 'Node') tensor(1.0000e-06) tensor(4.3898)
('Node', 'def_and_ball', 'Node') tensor(1.0000e-06) tensor(3.4072)
('Node', 'temporal', 'Node') tensor(1.) tensor(1.)


In [ ]:
# 1) Node feature 전체 꺼내기
node_feats = sample["graph"]["Node"].x    # shape: [총노드수, 특성차원]

# 2) 보고 싶은 노드의 인덱스(index) 지정
# 예를 들어, 첫 번째 프레임의 0번 공격수 노드를 보고 싶다면
nodes_per_frame = 23   # 공격수11 + 수비수11 + 볼1
frame_idx = 10          # 0번째 프레임
player_idx = 9      # 프레임 내에서 0번 노드
node_idx = frame_idx * nodes_per_frame + player_idx

# 3) 해당 노드만 출력
single = node_feats[node_idx]              # tensor of shape [특성차원]
print(single)


tensor([ 1.2305e-01,  2.2853e-01,  1.7007e-02, -4.4118e-02,  6.7407e+01,
         1.1000e+01,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00])


In [ ]:
sample["condition"].shape

torch.Size([125, 202])

In [ ]:
sample["condition_columns"][:9]

['Away_22_x',
 'Away_22_y',
 'Away_22_vx',
 'Away_22_vy',
 'Away_22_dist',
 'Away_22_position',
 'Away_22_starter',
 'Away_22_possession_duration',
 'Away_22_neighbor_count']

In [ ]:
bases = []
for col in sample["condition_columns"]:
    if col.startswith("ball_"):
        continue
    parts = col.split("_", 2)
    base = "_".join(parts[:2])
    if base not in bases:
        bases.append(base)

In [ ]:
bases

['Away_22',
 'Away_25',
 'Away_26',
 'Away_29',
 'Away_31',
 'Away_33',
 'Away_34',
 'Away_35',
 'Away_36',
 'Away_37',
 'Away_39',
 'Home_2',
 'Home_3',
 'Home_5',
 'Home_6',
 'Home_8',
 'Home_9',
 'Home_11',
 'Home_12',
 'Home_13',
 'Home_14',
 'Home_17']